# <center>SPY Classification with Intraday Data Based Variables</center>

For this study, we are going to use classification to assign 'Buy', 'Flat', or 'Sell' to each row of data.

The data is one day of SPY values.

The difference of this study vs my peers is that I am focusing on volume and exploring its value in classifying the data. 
This study includes values that are derived from intraday data, in addition to daily data.  
Actually, the primary focus are the variables that are derived from the intraday data and their impact on the variables derived from there.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mlUtilities import ml_utils, data_utils

In [2]:
from importlib import reload

# Load Data

<hr size='3'>

Our primary dataframes will be i_data and d_data--i for intraday and d for daily.
Both variables are pandas DataFrame objects

### Intraday Data Load and Preparation

In [3]:
i_data = pd.read_csv('Data/5m_SPY')

In [6]:
i_data.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2021-11-17 09:30:00-05:00,469.000000,469.040009,468.450012,468.519989,468.519989,2523699
1,2021-11-17 09:35:00-05:00,468.529999,468.540009,468.160004,468.320007,468.320007,836314
2,2021-11-17 09:40:00-05:00,468.320007,468.524994,468.160004,468.450012,468.450012,577620
3,2021-11-17 09:45:00-05:00,468.450012,468.640015,468.130005,468.135010,468.135010,865588
4,2021-11-17 09:50:00-05:00,468.135010,468.135010,467.579987,467.670013,467.670013,951646


In [5]:
i_data['Datetime'] = pd.to_datetime(i_data['Datetime'])